<a href="https://colab.research.google.com/github/hecaguayo/Tarea1MLFBI/blob/master/Tarea_1_MLBI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Tarea 1 de MLBI 2020-1**
Hector Aguayo & Fabián Segura

# IMPORTACION DE MODULOS REQUERIDOS

Se importan numpy, pandas, tensorflow, etc

In [126]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import logging
import numpy as np

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger("tensorflow").setLevel(logging.WARNING)

from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(2)

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.2.0-rc3


Se deben leer los datos de los dataset envíados, el primero es un archivo .xls

# LECTURA DE DATOS

Se abre archivo desde Github y se importan usando pandas

In [0]:
url = 'https://github.com/FxknTT/TareaMLBI1/blob/master/dataset1.xlsx?raw=true'
dataset = pd.read_excel(url)

Se verifica que ningun dato falte

In [128]:
dataset.isna().sum()

crr_Nom                     0
psu_Nem                     0
psu_Leng                    0
psu_Mate                    0
psu_Cie                     0
psu_Pond                    0
Longitud                    0
Latitud                     0
cred_apr                    0
distancia                   0
F                           0
M                           0
PARTICULAR_SUBVENCIONADO    0
PARTICULAR_PAGADO           0
MUNICIPAL                   0
cluster                     0
dtype: int64

# LIMPIEZA DE DATOS

Se debe alertar que "crr_Nom" es categorica, no numerica. Se convierte a un "one hot"

Cambiando las carrearas a filas one hot (variables dummy)

In [0]:
carreras = pd.get_dummies(dataset['crr_Nom'],prefix="Carrera")

Ahora obtenemos una nueva tabla llamada carrera en donde nos indica con valores 0 o 1 si pertenece o no a alguna Ingeniería.

In [130]:
carreras.head() 

,Carrera_INGENIERIA CIVIL,Carrera_INGENIERIA CIVIL - PLAN COMUN,Carrera_INGENIERIA CIVIL AEROESPACIAL,Carrera_INGENIERIA CIVIL BIOMEDICA,Carrera_INGENIERIA CIVIL DE MATERIALES,Carrera_INGENIERIA CIVIL ELECTRICA,Carrera_INGENIERIA CIVIL ELECTRONICA,Carrera_INGENIERIA CIVIL EN TELECOMUNICACIONES,Carrera_INGENIERIA CIVIL INDUSTRIAL,Carrera_INGENIERIA CIVIL INFORMATICA,Carrera_INGENIERIA CIVIL MECANICA,Carrera_INGENIERIA CIVIL METALURGICA,Carrera_INGENIERIA CIVIL QUIMICA,Carrera_INGENIERÍA CIVIL DE MINAS
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Ahora juntamos nuestra dataset original con la nueva tabla obtenida.

In [0]:
dataset = pd.concat([dataset, carreras], axis=1)

In [132]:
dataset.head()

,crr_Nom,psu_Nem,psu_Leng,psu_Mate,psu_Cie,psu_Pond,Longitud,Latitud,cred_apr,distancia,F,M,PARTICULAR_SUBVENCIONADO,PARTICULAR_PAGADO,MUNICIPAL,cluster,Carrera_INGENIERIA CIVIL,Carrera_INGENIERIA CIVIL - PLAN COMUN,Carrera_INGENIERIA CIVIL AEROESPACIAL,Carrera_INGENIERIA CIVIL BIOMEDICA,Carrera_INGENIERIA CIVIL DE MATERIALES,Carrera_INGENIERIA CIVIL ELECTRICA,Carrera_INGENIERIA CIVIL ELECTRONICA,Carrera_INGENIERIA CIVIL EN TELECOMUNICACIONES,Carrera_INGENIERIA CIVIL INDUSTRIAL,Carrera_INGENIERIA CIVIL INFORMATICA,Carrera_INGENIERIA CIVIL MECANICA,Carrera_INGENIERIA CIVIL METALURGICA,Carrera_INGENIERIA CIVIL QUIMICA,Carrera_INGENIERÍA CIVIL DE MINAS
0,INGENIERIA CIVIL INDUSTRIAL,783,653,685,672,735,-73.16487,-36.978759,38,20.201244,1,0,1,0,0,C2,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,INGENIERIA CIVIL BIOMEDICA,799,705,713,686,756,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,INGENIERIA CIVIL QUIMICA,688,543,612,534,638,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C3,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,INGENIERIA CIVIL ELECTRICA,605,598,658,601,633,-73.16487,-36.978759,21,20.201244,0,1,1,0,0,C2,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,INGENIERIA CIVIL INFORMATICA,603,710,668,651,657,-73.16487,-36.978759,19,20.201244,0,1,1,0,0,C2,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Ahora procedemos a quitar la variale Crr_Nom

In [0]:
dataset = dataset.drop(["crr_Nom"],axis=1)

Revisamos nuestras variables

In [134]:
dataset.isna().sum() #Revisamos que todas las variables tengan datos.

psu_Nem                                           0
psu_Leng                                          0
psu_Mate                                          0
psu_Cie                                           0
psu_Pond                                          0
Longitud                                          0
Latitud                                           0
cred_apr                                          0
distancia                                         0
F                                                 0
M                                                 0
PARTICULAR_SUBVENCIONADO                          0
PARTICULAR_PAGADO                                 0
MUNICIPAL                                         0
cluster                                           0
Carrera_INGENIERIA CIVIL                          0
Carrera_INGENIERIA CIVIL - PLAN COMUN             0
Carrera_INGENIERIA CIVIL AEROESPACIAL             0
Carrera_INGENIERIA CIVIL BIOMEDICA                0
Carrera_INGE

**Ahora procedemos a realizar los mismos pasos anteriores con la variables CLUSTER.**

In [135]:
dataset.head()

,psu_Nem,psu_Leng,psu_Mate,psu_Cie,psu_Pond,Longitud,Latitud,cred_apr,distancia,F,M,PARTICULAR_SUBVENCIONADO,PARTICULAR_PAGADO,MUNICIPAL,cluster,Carrera_INGENIERIA CIVIL,Carrera_INGENIERIA CIVIL - PLAN COMUN,Carrera_INGENIERIA CIVIL AEROESPACIAL,Carrera_INGENIERIA CIVIL BIOMEDICA,Carrera_INGENIERIA CIVIL DE MATERIALES,Carrera_INGENIERIA CIVIL ELECTRICA,Carrera_INGENIERIA CIVIL ELECTRONICA,Carrera_INGENIERIA CIVIL EN TELECOMUNICACIONES,Carrera_INGENIERIA CIVIL INDUSTRIAL,Carrera_INGENIERIA CIVIL INFORMATICA,Carrera_INGENIERIA CIVIL MECANICA,Carrera_INGENIERIA CIVIL METALURGICA,Carrera_INGENIERIA CIVIL QUIMICA,Carrera_INGENIERÍA CIVIL DE MINAS
0,783,653,685,672,735,-73.16487,-36.978759,38,20.201244,1,0,1,0,0,C2,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,799,705,713,686,756,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,688,543,612,534,638,-73.16487,-36.978759,36,20.201244,1,0,1,0,0,C3,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,605,598,658,601,633,-73.16487,-36.978759,21,20.201244,0,1,1,0,0,C2,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,603,710,668,651,657,-73.16487,-36.978759,19,20.201244,0,1,1,0,0,C2,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Creamos una variable y, que tiene los valores que deseamos predecir.

In [136]:
y = pd.get_dummies(dataset['cluster'],prefix="CLUSTER")
print(y)

      CLUSTER_C1  CLUSTER_C2  CLUSTER_C3  CLUSTER_C4
0              0           1           0           0
1              1           0           0           0
2              0           0           1           0
3              0           1           0           0
4              0           1           0           0
...          ...         ...         ...         ...
7861           0           1           0           0
7862           0           1           0           0
7863           0           0           1           0
7864           0           1           0           0
7865           0           0           0           1

[7866 rows x 4 columns]


Por último quitamos la variable cluster de nuestro dataset.



In [0]:
dataset = dataset.drop(["cluster"],axis=1)

In [138]:
print(dataset.isna().sum())
print(y.isna().sum())

psu_Nem                                           0
psu_Leng                                          0
psu_Mate                                          0
psu_Cie                                           0
psu_Pond                                          0
Longitud                                          0
Latitud                                           0
cred_apr                                          0
distancia                                         0
F                                                 0
M                                                 0
PARTICULAR_SUBVENCIONADO                          0
PARTICULAR_PAGADO                                 0
MUNICIPAL                                         0
Carrera_INGENIERIA CIVIL                          0
Carrera_INGENIERIA CIVIL - PLAN COMUN             0
Carrera_INGENIERIA CIVIL AEROESPACIAL             0
Carrera_INGENIERIA CIVIL BIOMEDICA                0
Carrera_INGENIERIA CIVIL DE MATERIALES            0
Carrera_INGE

Finalmente tenemos nuestro dataset con sólo variables numericas y además nuestra matriz y con los valores que desaemos predecir.

# DEFINIENDO RANGOS DE ENTRENAMIENTO Y TESTEO

Primero realizamos un cast a nuestro dataset y nuestra tabla y, transformando ambas en un dtype=tf.float64

In [0]:
dataset = tf.cast(dataset, dtype=tf.float64)
y = tf.cast(y, dtype=tf.float64)

Revisamos que se haya realizado correctamente

In [163]:
print(dataset)
print(y)

tf.Tensor(
[[783. 653. 685. ...   0.   0.   0.]
 [799. 705. 713. ...   0.   0.   0.]
 [688. 543. 612. ...   0.   1.   0.]
 ...
 [641. 598. 640. ...   0.   0.   0.]
 [641. 653. 595. ...   0.   0.   0.]
 [641. 524. 554. ...   0.   0.   0.]], shape=(7866, 28), dtype=float64)
tf.Tensor(
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]], shape=(7866, 4), dtype=float64)


Ahora dividimos nuestros datos en un 80% den entrenamiento y un 20% en testeo.

In [164]:
data_size = len(dataset)
train_pct = 0.8

train_size = int(data_size * train_pct)
print(train_size)

x_train, y_train = dataset[:train_size], y[:train_size]
x_test, y_test = dataset[train_size:], y[train_size:]

6292


In [165]:
x_train.shape[0], x_test.shape[0]

(6292, 1574)

Revisamos

In [166]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

tf.Tensor(
[[783. 653. 685. ...   0.   0.   0.]
 [799. 705. 713. ...   0.   0.   0.]
 [688. 543. 612. ...   0.   1.   0.]
 ...
 [661. 626. 759. ...   0.   0.   0.]
 [661. 620. 708. ...   0.   0.   0.]
 [599. 547. 694. ...   0.   0.   0.]], shape=(6292, 28), dtype=float64)
tf.Tensor(
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]], shape=(6292, 4), dtype=float64)
tf.Tensor(
[[661. 542. 637. ...   1.   0.   0.]
 [608. 560. 648. ...   1.   0.   0.]
 [554. 660. 664. ...   1.   0.   0.]
 ...
 [641. 598. 640. ...   0.   0.   0.]
 [641. 653. 595. ...   0.   0.   0.]
 [641. 524. 554. ...   0.   0.   0.]], shape=(1574, 28), dtype=float64)
tf.Tensor(
[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]], shape=(1574, 4), dtype=float64)


# CREACION DE MODELO

Definición de layers

Modelo

Acá es para usar lo de tensorboard, pero no sé como funciona en Colab

In [0]:
logdir = "logs\\tf_keras\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

El modelo es de un solo layery es lineal

In [0]:
linear_layer = (units=1, input_shape=(28,))
model = tf.keras.models.Sequential([linear_layer])
model.compile(
    loss='mse', # keras.lotf.keras.layers.Densesses.mean_squared_error
    optimizer=keras.optimizers.SGD(lr=0.2),
)


Sumario para ver las variables

In [169]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 1)                 29        
Total params: 29
Trainable params: 29
Non-trainable params: 0
_________________________________________________________________


Se entrenan las variables

In [170]:
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size = train_size,
    verbose = 1, # Suppress chatty output; use Tensorboard instead
    epochs = 100,
    validation_data = (x_test, y_test),
    callbacks=[tensorboard_callback]
)

Epoch 1/100
1/1 [==============================] - 0s 69ms/step - loss: 13720.1641 - val_loss: 9138030401028096.0000
Epoch 2/100
1/1 [==============================] - 0s 33ms/step - loss: 9052687253372928.0000 - val_loss: 6543168737373883960035115008.0000
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 6481186496694599149194051584.0000 - val_loss: inf
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: inf - val_loss: inf
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: inf - val_loss: inf
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: inf - val_loss: inf
Epoch 7/100
1/1 [==============================] - 0s 40ms/step - loss: inf - val_loss: nan
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: nan - val_loss: nan
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: nan - val_loss: nan
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - lo

Ver como se entrenaron los pesos